<a href="https://colab.research.google.com/github/CasualMathEnjoyer/01RAD_Ex09_HW/blob/main/code/01RAD_Ex09_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD  Exercise 09 - Homework

Authors: Your names here  
Date: 2024-12-03  

---

## Task Description

The dataset is based on the **House Sales in King County, USA** dataset, which can be found, for example, on kaggle.com or in the `moderndive` library under the name `house_prices`. The original dataset contains house sale prices in the King County area, which includes Seattle, and the data was collected between May 2014 and May 2015. For our purposes, several variables have been removed, and the dataset has been significantly reduced and slightly modified.

The dataset has already been split into three parts and modified, all of which will be used progressively throughout this assignment.

---

## Variables Description

The dataset contains the following 18 variables, and our goal is to explore the influence of 12 of them on the target variable `price`.

| Feature         | Description                                           |
|------------------|-------------------------------------------------------|
| `id`            | Unique identifier for a house                         |
| `price`         | Sale price (prediction target)                        |
| `bedrooms`      | Number of bedrooms                                    |
| `bathrooms`     | Number of bathrooms                                   |
| `sqft_living`   | Square footage of the home                            |
| `sqft_lot`      | Square footage of the lot                             |
| `floors`        | Total number of floors (levels) in the house          |
| `waterfront`    | Whether the house has a waterfront view               |
| `view`          | Number of times the house has been viewed             |
| `condition`     | Overall condition of the house                        |
| `grade`         | Overall grade given to the housing unit               |
| `sqft_above`    | Square footage of the house apart from the basement   |
| `sqft_basement` | Square footage of the basement                        |
| `yr_built`      | Year the house was built                              |
| `yr_renovated`  | Year when the house was renovated                     |
| `sqft_living15` | Living room area in 2015 (after renovations)          |
| `sqft_lot15`    | Lot size in 2015 (after renovations)                  |
| `split`         | Splitting variable with train, test, and validation samples |

---


In [ ]:
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/francji1/01RAD/main/data/01RAD_2024_house.csv"
house_rad = pd.read_csv(url)

# Display the first few rows of the dataset
house_rad.head()



---

## Exploratory and Graphical Analysis

### Question 1

Verify the dimensions of the dataset, the types of individual variables, and summarize the basic descriptive statistics of all variables. Plot a histogram and a density estimate for the target variable `price`. Can anything be inferred for future analysis?

---

### Question 2

Are all variables usable for analysis and prediction of house prices? If the data contains missing values (or strange or nonsensical observations), can they be imputed (corrected), or must they be removed from the dataset?

---

### Question 3

For the selected variables (`price`, `sqft_living`, `grade`, `yr_built`), verify whether the split into train, test, and validation datasets was random. That is, do these variables have approximately the same distributions across the train, test, and validation groups?

---


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# QUESTION 1

# Verify the dimensions of the dataset
print("\nDimensions of the dataset:", house_rad.shape)

# Data types of individual variables
print("\nData types of variables:")
print(house_rad.dtypes)

# Summarize basic descriptive statistics of all variables
print("\nDescriptive statistics:")
print(house_rad.describe())


**Features with Potential Outliers:**

  - Bathrooms: The maximum value (275) is abnormally high and likely an outlier.
  - Grade: The maximum value (232) is unusually high compared to the 75th percentile (8), suggesting extreme outliers.
  - Sqft_living and Sqft_lot: The large maximum values (33,600 and 641,203, respectively) compared to their means and 75th percentiles suggest outliers.

**Missing or Sparse Data:**

  - Waterfront: The mean value is 0.021, indicating that only about 2.1% of houses have a waterfront view.
  - Yr_renovated: Most houses likely have not been renovated, as the median value is 0, which might represent missing or unrenovated properties.

**Feature Ranges and Potential Relationships:**

  - Bedrooms and Bathrooms: These are relatively stable in their ranges, with typical values between 2-6 bedrooms and 1-4 bathrooms, excluding outliers.
  - Sqft_living and Grade: These have broader ranges and may be closely correlated with price since they represent home size and quality.
  - Year Built: The range (1900-2015) suggests a mix of old and modern homes.

---



# Question 1


In [ ]:
# Plot histogram and density estimate for the target variable price
plt.figure(figsize=(12, 6))
house_rad['price'].hist(bins=100, alpha=0.7, label='Histogram', density=True)
house_rad['price'].plot(kind='kde', label='Density Estimate', color='red')
plt.title('Distribution of Sale Price')
plt.xlabel('Price')
plt.ylabel('Frequency/Density')
plt.legend()
plt.show()

The histogram suggest that the price distribution is right-skewed, with most prices concentrated in the lower range but a long tail extending to very high prices.

In [ ]:
# Select a subset of columns to visualize (price and continuous variables)
columns_to_plot = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
                   'floors', 'view', 'sqft_above', 'sqft_basement',
                   'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']

# Create pairplot
sns.pairplot(house_rad[columns_to_plot], diag_kind='hist', height=2.5, hue='yr_renovated')
plt.show()

In [ ]:
# Plot histograms for all columns
house_rad.hist(bins=30, figsize=(15, 10), grid=False, edgecolor='black', alpha=0.7)
plt.tight_layout()
plt.show()

# Question 2


In [ ]:
# QUESTION 2

# Check for missing values in each column
missing_values = house_rad.isnull().sum()

# Display columns with missing values
columns_with_nans = missing_values[missing_values > 0]

print("Columns with missing values and their counts:")
print(columns_with_nans)


There aren't any variables with missing values.

Variable improtance:


In [ ]:
# Select only numeric columns
numeric_columns = house_rad.select_dtypes(include=['number'])

# Compute the correlation matrix
correlation_matrix = numeric_columns.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Correlation Heatmap')
plt.show()



Correlation:
- price and sqft_living, sqft_above, sft_living14
- price view, waterfront, floors, bedrooms, yr_renovated
- svarious sqft correlations (as expected)

Negative correlation:
- year build vs year renovated (as expected)


## Linear Model (Use Only Training Data, i.e., split == "train")

### Question 4

Calculate the correlations between the regressors and visualize them. Also, compute the condition number (Kappa) and the variance inflation factor (VIF). If multicollinearity is present, decide which variables to use and justify your choices.

---

### Question 5

Using only the training data (split == "train") and all selected variables, find a suitable linear regression model that best predicts the target variable `price`, i.e., minimizes the mean squared error (MSE). Compare the VIF and Kappa values of the final model to those of the original regressor matrix.

---

### Question 6

For your selected model from the previous question, calculate the relevant influence measures. Provide the `id` for the top 20 observations with the highest values of DIFF, the highest leverage (hat values), and the highest Cook’s distance (i.e., 3 sets of 20 values). Which observations do you consider influential or outliers?

---

### Question 7

Validate the model graphically using residual plots (Residual vs. Fitted, QQ-plot, Cook’s distance, leverage, etc.). Based on this and the previous question, did you identify any suspicious observations in the data that might have resulted from dataset adjustments? Would you recommend removing these observations from the data?

---


# Question 5


In [ ]:
def model_Q5(data, selected_features, split):
    print("Model for data split:", split)
    split_data = data[data['split'] == split]
    X = split_data[selected_features]
    y = split_data['price']

    # Handle missing values
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())

    # # Create interaction terms
    # X['sqft_living_grade'] = X['sqft_living'] * X['grade']
    # X['bathrooms_bedrooms'] = X['bathrooms'] * X['bedrooms']

    # Add a constant for statsmodels regression
    X_with_const = sm.add_constant(X)

    # Fit the regression model
    model = sm.OLS(y, X_with_const).fit()

    # Print the summary
    print(model.summary())

    # Calculate Mean Squared Error (MSE)
    y_pred = model.predict(X_with_const)
    mse = ((y - y_pred) ** 2).mean()

    # Calculate kappa (condition number of the design matrix)
    kappa = np.linalg.cond(X_with_const)

    # Calculate VIF for each feature
    vif_data = pd.DataFrame({
        "Variable": X_with_const.columns,
        "VIF": [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])]
    })

    # Display results
    print("\nVariance Inflation Factors (VIF):")
    print(vif_data)
    print(f"\nMean Squared Error (MSE): {mse}")
    print(f"Kappa (Condition Number): {kappa}")
    print('\n')


In [ ]:
selected_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
                     'floors', 'waterfront', 'view', 'condition',
                     'grade', 'sqft_above', 'sqft_basement',
                     'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']
selected_features = ['sqft_living',
                     'floors', 'view',
                     'sqft_basement',
                     'yr_renovated']


model_Q5(house_rad, selected_features, split='train')


## Robust Approach

### Question 8

If you decided to remove any observations from the dataset, work with the filtered dataset, retrain the model from Question 5, and calculate the $R^2$ statistic and MSE on both the training and test data (split == "test").

---

### Question 9

Estimate the regression coefficients using a robust method, such as Total Least Squares (TLS), on both the filtered dataset (after removing any observations, if applicable) and the original full dataset. Compare the results and discuss any differences in the estimated coefficients and model performance. What insights can you draw about the impact of filtering observations on model robustness?

---
### Question 10

Explore additional robust regression approaches (e.g., Huber regression, quantile regression, or M-estimators) to estimate the regression coefficients on the filtered and full datasets. Compare the results across these methods, discussing the strengths and limitations of each approach in the context of predicting house prices in the King County area. How do these robust methods handle potential outliers or influential observations in the data?

---
### Question 11

Select the final model and compare the MSE and $R^2$ on the training, test, and validation datasets. What do these values suggest about the quality of the model and potential overfitting? Is your model suitable for predicting house prices in the King County area? If so, does this prediction have any limitations?



# Question 8

In [ ]:
# Display rows with extreme values for bathrooms


outliers = house_rad[house_rad['bathrooms'] > 5]
print(outliers.shape)

outliers = house_rad[house_rad['sqft_living'] < house_rad['sqft_above']]
print(outliers.shape)

outliers = house_rad[house_rad['sqft_living'] < house_rad['sqft_basement']]
print(outliers.shape)

outliers = house_rad[house_rad['sqft_lot'] < house_rad['sqft_above']]
print(outliers.shape)

outliers = house_rad[house_rad['sqft_lot'] < house_rad['sqft_basement']]
print(outliers.shape)

# outliers = house_rad[house_rad['sqft_lot'] < house_rad['sqft_living']]  # doesn't necesarily logically need to be an outlier - building can be vertical
# print(outliers.shape)

# outliers = house_rad[house_rad['sqft_lot15'] < house_rad['sqft_living15']]
# print(outliers.shape)

outliers = house_rad[house_rad['sqft_lot'] < 100]
print(outliers.shape)

outliers = house_rad[(house_rad['sqft_living'] < 500) & (house_rad['bathrooms'] > 2)]  # not added condition, just seemed weird
print(outliers.shape)

# Define conditions
conditions = (
    (house_rad['bathrooms'] <= 5)
    & (house_rad['sqft_lot'] > 100)
    & (house_rad['sqft_living'] >= house_rad['sqft_above'])
    & (house_rad['sqft_living'] >= house_rad['sqft_basement'])
    & (house_rad['sqft_lot'] >= house_rad['sqft_above'])
    & (house_rad['sqft_lot'] >= house_rad['sqft_basement'])
    # & (house_rad['sqft_lot'] >= house_rad['sqft_living'])
    # & (house_rad['sqft_lot15'] >= house_rad['sqft_living15'])
)

# Apply the conditions to filter the dataset
filtered_data = house_rad[conditions]

# Print dataset sizes
print(f"Original dataset size: {house_rad.shape}")
print(f"Filtered dataset size after manual removal: {filtered_data.shape}")



In [ ]:
# test on model from question 5
model_Q5(filtered_data, selected_features, split="train")
model_Q5(filtered_data, selected_features, split="test")

---

## Voluntary part: Machine Learning Approach


### Question 12
Apply machine learning-based linear regression methods, such as Ridge Regression, Lasso, or Elastic Net, to the dataset. Use the train-test split to evaluate model performance and focus on feature selection. Identify the most relevant features based on these methods and compare how the selected features impact the model's predictive performance. Discuss how regularization affects feature selection and the trade-offs between bias and variance in the context of house price prediction. Additionally, evaluate the stability of selected features across different methods and provide recommendations for choosing the optimal feature set.
